In [5]:
import pandas as pd
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
import random

from tqdm import tqdm
tqdm.pandas()

In [6]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# BASE_PATH = '/kaggle/input/gen-ai-ucu-2024-task-3'
BASE_PATH = 'data'

In [7]:
train_df = pd.read_json(f"{BASE_PATH}/zno.train.jsonl", lines=True)
test_df = pd.read_json(f"{BASE_PATH}/zno.test.jsonl", lines=True)

In [8]:
# "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
# "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
MODEL_NAME = "unsloth/Qwen2.5-7B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 2048,
    dtype = None, # None for auto detection.
    load_in_4bit = True, # 4bit quantization to reduce memory usage
)

==((====))==  Unsloth 2025.1.5: Fast Qwen2 patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # Rank stabilized LoRA is supported
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
zno_prompt = """Below is a question about Ukrainian history, language and literature. Select the correct answer from the provided options.

### Question:
{}

### Options:
{}

### Correct Answer:
{}"""

In [11]:
def solve_task(row):
    question = row['question']
    options = ','.join([f"[{option['marker']}] {option['text']}" for option in row['answers']])

    inputs = tokenizer([zno_prompt.format(question, options, "",)], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=3, use_cache=True)
    outputs = tokenizer.batch_decode(outputs)
    result = outputs[0].split("Correct Answer:\n")[1]
    return [result[1]]

In [12]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    zno_prompt.format(
        'Позначте рядок, у якому в усіх словах потрібно писати літеру *и*', # instruction
        '(А) бад..лина, благоч..стивий, кр..хкий, ж..виця;,(Б) вар..во, меж..річчя, вич..пурений, кр..шталь;,(В) п’ят..річка, заруч..ни, нев..димка, обітн..ця;,(Г) зач..нати, виконав..ця, знів..чити, вел..чина;,(Д) нож..чок, печ..во, викор..нити, оз..ратися.', # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 3, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is a question about Ukrainian history, language and literature. Select the correct answer from the provided options.\n\n### Question:\nПозначте рядок, у якому в усіх словах потрібно писати літеру *и*\n\n### Options:\n(А) бад..лина, благоч..стивий, кр..хкий, ж..виця;,(Б) вар..во, меж..річчя, вич..пурений, кр..шталь;,(В) п’ят..річка, заруч..ни, нев..димка, обітн..ця;,(Г) зач..нати, виконав..ця, знів..чити, вел..чина;,(Д) нож..чок, печ..во, викор..нити, оз..ратися.\n\n### Correct Answer:\n(В)']

In [13]:
submission_df = test_df.copy()
submission_df['correct_answers'] = test_df.progress_apply(solve_task, axis=1)
submission_df

100%|██████████| 751/751 [02:17<00:00,  5.47it/s]


,question,answers,subject,id,correct_answers
0,«Сміхом крізь сльози» можна схарактеризувати з...,"[{'marker': 'А', 'text': '«Три зозулі з поклон...",ukrainian-language-and-literature,0,[Б]
1,"Удовин син, мати, сестра, кохана – ключові обр...","[{'marker': 'А', 'text': '«Засвіт встали козач...",ukrainian-language-and-literature,1,[В]
2,В уривку з історичного джерела «*Створивши бан...,"[{'marker': 'А', 'text': 'Правобережної Україн...",history-of-ukraine,2,[А]
3,В уривку\n\n\n*Доки буде жити Україна\n\nВ теп...,"[{'marker': 'А', 'text': 'Василя Стефаника'}, ...",ukrainian-language-and-literature,3,[Д]
4,Букву ***и*** на місці пропуску треба писати в...,"[{'marker': 'А', 'text': 'пр….хований, пр…звис...",ukrainian-language-and-literature,4,[В]
...,...,...,...,...,...
746,Укажіть правильний варіант послідовного заповн...,"[{'marker': 'А', 'text': 'дієвих прийомів, які...",ukrainian-language-and-literature,746,[Г]
747,**Проаналізуйте фрагмент історичного документа...,"[{'marker': 'А', 'text': 'Українська головна в...",history-of-ukraine,747,[Г]
748,Прочитайте речення *(цифра позначає наступне с...,"[{'marker': 'А', 'text': '3, 4, 5, 10'}, {'mar...",ukrainian-language-and-literature,748,[В]
749,Граматично правильне продовження речення «*Пер...,"[{'marker': 'А', 'text': 'мені пригадалися дав...",ukrainian-language-and-literature,749,[А]


In [14]:
submission_df['correct_answers'] = submission_df['correct_answers'].apply(lambda x: x[0])
submission_df[["id", "correct_answers"]].to_csv("submission-baseline.csv", index=False)